In [1]:
import awswrangler as wr
import pandas as pd

from data_sci_toolkit.aws_tools import permission_tools 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

/opt/miniconda3/envs/stellar/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
SESSION = permission_tools.get_aws_session("Dev-DataScienceAdmin")

Authorized as AROAQBUNWK7XN5DTYWPLJ:rkazmerik@stellaralgo.com


In [3]:
DATABASE = "integrations_milb_aviators"
S3_BUCKET = "s3://dev-athena-testing-us-east-1"

In [4]:
df = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        SELECT * FROM integrations_milb_aviators.tickets LIMIT 10 
    """
)
df.shape

(10, 26)

In [5]:
df

,source_system,source_system_group,product,client_property,seat_id,section_name,row_name,seat_num,season_id,event_id,event_name,plan_event_id,ticket_type,comp_code,promo_code,account_id,price_code,price,account_rep_id,account_rep_name,purchased_at,returned_at,updated_at,forwards,resales,scans
0,ticketmaster,ticketing,Individual,milb_aviators,AF9B5B10787E4E003F73D8DA07A33795,DOB115,L,17,55,2860,EV200409,<NA>,Adult,0,<NA>,1317610,E*,25.000000,28,Bryan Frey,2020-02-14 05:22:15.210,2020-07-14 07:02:06.003,2020-02-22 09:04:56,"[{'source_account_id': '1317610', 'account_id'...",None,None
1,ticketmaster,ticketing,Full Season,milb_aviators,11B3459BA583025EF767E1359302E403,HPB110,Q,16,55,2929,EV200907,2930,Adult,0,<NA>,101400,CSCH,1375.000000,1242776,David Moses,2019-08-20 19:47:18.132,2020-10-07 10:24:43.707,2020-01-30 11:56:38,None,None,None
2,ticketmaster,ticketing,Mini Plan,milb_aviators,935C3C1E55CB70F9783C4DA943E336A3,DOB118,B,10,55,2864,EV200413,2939,Dugout Box,0,<NA>,1269985,ESQ,18.590000,1199851,Nathan Erbach,2019-08-13 15:19:21.288,2020-10-05 10:51:32.087,2019-12-08 02:02:48,None,None,None
3,ticketmaster,ticketing,Half Season,milb_aviators,D205DF6724C6AEB7A67E21800801C1F4,DOB114,J,7,55,2912,EV200805,2931,Dugout Box,0,<NA>,593260,ESH,17.289999,1213561,Ariel Greenberg,2019-08-13 14:47:36.338,2020-11-04 12:20:33.422,2020-02-25 16:50:15,None,None,None
4,ticketmaster,ticketing,Full Season,milb_aviators,BA7A63066240F946C5B2AD3C4DAAEB90,DOB115,N,15,55,2871,EV200427,2930,Dugout Box,0,<NA>,1764449,ESF,1100.000000,16,Erik Eisenberg,2020-02-14 08:56:06.666,2020-06-30 21:21:05.354,2020-02-14 08:56:54,None,None,None
5,ticketmaster,ticketing,Full Season,milb_aviators,7D460CB21959B6F621EFB6714AECD45F,DOP114,G,6,55,2896,EV200625,2930,Adult,0,<NA>,1243634,DSCH,19.650000,1199851,Nathan Erbach,2019-08-20 19:49:02.897,2020-10-07 10:35:56.206,2020-01-30 12:10:36,None,None,None
6,ticketmaster,ticketing,Full Season,milb_aviators,D49F0012035656CDD36440CCBFF8E067,CL211,C,10,55,2878,EV200514,2930,Adult,0,<NA>,1720892,KSFN,2500.000000,28,Bryan Frey,2020-07-27 11:44:31.416,2020-10-07 10:10:03.741,2020-07-27 11:44:31,None,None,None
7,ticketmaster,ticketing,Group,milb_aviators,00D314C5D908EEE21D64CF7A08F61263,3RDPD1,GA1,48,55,2861,EV200410,<NA>,Group Full NO DISC,0,<NA>,337873,NG,17.000000,16,Erik Eisenberg,2020-02-20 20:59:42.593,2020-06-01 12:04:59.397,2020-02-24 08:26:24,None,None,None
8,ticketmaster,ticketing,Half Season,milb_aviators,B8D9FC5EE1F3C52F13864D39B38ECA24,HPB112,K,14,55,2885,EV200530,2932,Home Plate Box,0,<NA>,1765229,CSH,21.230000,28,Bryan Frey,2020-02-24 17:35:24.542,2020-10-07 11:45:28.029,2020-02-24 17:35:24,None,None,None
9,ticketmaster,ticketing,Full Season,milb_aviators,5E661ED51FB6FCB6A6D3714D516697FE,DOB114,L,2,55,2928,EV200906,2930,Adult,0,<NA>,556217,ESCH,16.080000,16,Erik Eisenberg,2019-08-20 19:50:10.422,2020-10-07 10:32:49.025,2020-01-30 12:01:16,None,None,None


In [8]:
df2 = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        WITH customers as (
        select * from integrations_milb_aviators.customers
        CROSS JOIN UNNEST(account_ids['ticketing'], account_ids['retail']) as t(ticket_ids, retail_ids)
        ),
        ticketing AS (
        select *
        from integrations_milb_aviators.tickets t
        CROSS JOIN UNNEST(resales, forwards, scans) as t(resale_data, forward_data, scan_data)
        where returned_at IS NULL
        )
        select * from ticketing t
        INNER JOIN integrations_milb_aviators.ticketing_events e
        on t.event_id = e.id
        INNER JOIN customers c
        on c.ticket_ids = t.account_id limit 10 
    """
)
df2.shape

InvalidCtasApproachQuery: Please, define distinct names for your columns. Root error message: SYNTAX_ERROR: line 1:1: Column name 'source_system' specified more than once. You may need to manually clean the data at location 's3://dev-athena-testing-us-east-1/tables/a8150ddc-96d4-411f-a92c-82147aa9eb36' before retrying. Athena will not delete data in your account.